# 변형: Transforming

이 노트북에서는 언어 번역, 철자 및 문법 검사, 톤 조정 및 형식 변환과 같은 텍스트 변환 작업에 Large Language Model을 사용하는 방법에 대해 알아보겠습니다.

## 설정

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

## 번역하기

ChatGPT는 여러 언어로 된 소스를 사용하여 교육을 받습니다. 때문에, 모델은 번역 능력을 갖게됩니다. 다음은 이 기능을 사용하는 몇 가지 예입니다.

In [ ]:
prompt = f"""
다음 영어 텍스트를 스페인어로 번역하세요: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
이것이 어떤 언어인지 알려주세요: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
다음 텍스트를 프랑스어, 스페인어, 영국 해적어로 번역하세요: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
다음 텍스트를 격식체 와 비격식체 스페인어로 번역하세요: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

### 범용 번역기
당신이 대형 다국적 전자상거래 회사에서 IT를 담당하고 있다고 상상해보세요. 사용자는 IT 문제에 대해 다양한 언어로 메시지를 보냅니다. 당신의 직원들은 전 세계에서 왔고 자신들의 모국어만 구사합니다. 당신은 범용 번역기가 필요해요!

In [ ]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # 시스템 성능이 정상보다 느립니다
  "Mi monitor tiene píxeles que no se iluminan.",              # 모니터에 불이 들어오지 않는 픽셀이 있습니다
  "Il mio mouse non funziona",                                 # 마우스가 작동하지 않습니다
  "Mój klawisz Ctrl jest zepsuty",                             # 키보드에 제어 키가 고장났습니다
  "我的屏幕在闪烁"                                               # 화면이 깜박입니다
] 

In [ ]:
for issue in user_messages:
    prompt = f"이게 무슨 언어인지 알려주세요: ```{issue}```"
    lang = get_completion(prompt)
    print(f"원본 메시지 ({lang}): {issue}")

    prompt = f"""
    다음 텍스트를 영어와 한국어로 번역하세요: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

## 직접 해보세요!
번역을 직접 해보세요!

## 톤 변환
글은 독자에 따라 달라질 수 있다. ChatGPT는 다양한 어투를 구사할 수 있다.

In [ ]:
prompt = f"""
다음을 속어에서 비즈니스 레터로 번역하세요: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

## 형식 변환
ChatGPT는 형식 간 변환을 할 수 있습니다. 프롬프트에서 입력 및 출력 형식을 설명하면 됩니다.

In [ ]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
다음 python dictionary를 JSON에서 열 머리글과 제목이 있는 HTML 표로 변환합니다: {data_json}
"""
response = get_completion(prompt)
print(response)

In [ ]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

## 철자 검사/문법 검사

다음은 일반적인 문법과 철자 문제의 몇 가지 예와 LLM의 응답입니다. 

텍스트를 교정하기 위해 LLM에 신호를 보내려면 모델에게 '교정' 또는 '교정'을 지시합니다.

In [ ]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""다음 텍스트를 교정하고
        수정된 버전을 다시 작성하십시오. 
        오류가 발견되지 않으면 "오류를 찾을 수 없습니다."라고 말합니다. 
        텍스트 주변에 구두점 사용 안 함:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

In [ ]:
text = f"""
내 딸 생일 선물로 받은 거야 내 딸이 내 방에서 계속 내 것을 가져갔거든.  네, 어른들도 판다를 좋아해요.  그녀는 그것을 어디에나 가지고 다니는데, 그것은 매우 부드럽고 귀엽다.  한쪽 귀는 다른 한쪽 귀보다 약간 낮고, 저는 그것이 비대칭적으로 설계되었다고 생각하지 않습니다. 하지만 제가 지불한 것에 비해서는 조금 작습니다. 같은 가격에 더 큰 옵션이 있을 수도 있다고 생각합니다.  생각보다 하루 빨리 도착해서 딸에게 주기 전에 직접 가지고 놀게 되었어요.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

In [ ]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

In [ ]:
prompt = f"""
이 리뷰를 교정하고 수정합니다. 좀 더 설득력 있게. 
APA 스타일 가이드를 따르고 고급 판독기를 대상으로 합니다. 
마크다운 형식의 출력입니다.
텍스트: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

## 직접 해보세요!
사용자 자신의 검토를 구성하기 위해 지침을 변경해 보십시오.

다음 사이트를 참고했습니다:

https://writingprompts.com/bad-grammar-examples/
